# Add `forename`, `name`, and `gender` properties to `Person`

In [4]:
import ast
import spacy
from spacytei.train import batch_train
spacy.__version__

'2.0.18'

## load trainings material

* annotations done by Peter Andorfer
* using prodigy

In [5]:
file = "./training_data/invs_vn.jsonl"

In [6]:
with open(file) as f:
    TRAIN_DATA = f.readlines()

In [7]:
train_data = [ast.literal_eval(x) for x in TRAIN_DATA]
train_data[4]

['Pater Mayr Ampach [Mayr am Bach] (Webermeister), Bruneck, (Zeuge)',
 {'entities': [[0, 5, 'VN']]}]

## train a new model

* with label type `VN`

In [8]:
batch_train(train_data=train_data, output_dir='./data/names_model_vn', new_label='VN', eval_split=0.3, n_iter=8)

Created blank model from 'de'
226 train vs 97 test samples
Iteration Number: 0
Losses {'ner': 50.22393727048257}
Duration: 0:00:10.318900
p: 90.72164948453609; f: 91.19170984455958; r: 91.66666666666666
Saved model to: data\names_model_vn
######################
Iteration Number: 1
Losses {'ner': 32.54218516191098}
Duration: 0:00:10.765923
p: 94.79166666666666; f: 94.79166666666666; r: 94.79166666666666
Saved model to: data\names_model_vn
######################
Iteration Number: 2
Losses {'ner': 31.00159087994206}
Duration: 0:00:11.516956
p: 87.62886597938144; f: 88.08290155440415; r: 88.54166666666666
######################
Iteration Number: 3
Losses {'ner': 30.529981183131582}
Duration: 0:00:11.367421
p: 94.31818181818183; f: 90.21739130434783; r: 86.45833333333334
######################
Iteration Number: 4
Losses {'ner': 29.18159619653817}
Duration: 0:00:11.275460
p: 90.72164948453609; f: 91.19170984455958; r: 91.66666666666666
######################
Iteration Number: 5
Losses {'ner'

# annotate Persons

## forenames

In [9]:
nlp = spacy.load(r"./data/names_model_vn")

In [10]:
no_vn = Person.objects.filter(forename="")
no_vn.count()

16163

In [11]:
for x in Person.objects.all():
    doc = nlp("{}".format(x.written_name))
    for vn in doc.ents[:1]:
        if vn.label_ == "VN":
            x.forename = "{}".format(vn.text)
            break
    x.save()

## surnames

In [12]:
file = "./training_data/invs_nn.jsonl"

In [13]:
with open(file) as f:
    TRAIN_DATA = f.readlines()

In [15]:
train_data = [ast.literal_eval(x) for x in TRAIN_DATA]
train_data[4]
len(train_data)

870

In [17]:
batch_train(train_data=train_data, output_dir='./data/names_model_nn', new_label='NN', eval_split=0.2, n_iter=5)

Created blank model from 'de'
696 train vs 174 test samples
Iteration Number: 0
Losses {'ner': 125.3904237483417}
Duration: 0:00:34.221088
p: 95.51282051282051; f: 91.41104294478527; r: 87.6470588235294
Saved model to: data\names_model_nn
######################
Iteration Number: 1
Losses {'ner': 94.53152456315587}
Duration: 0:00:33.249036
p: 96.875; f: 93.93939393939394; r: 91.17647058823529
Saved model to: data\names_model_nn
######################
Iteration Number: 2
Losses {'ner': 84.56929500715343}
Duration: 0:00:33.850924
p: 95.83333333333334; f: 95.26627218934912; r: 94.70588235294117
Saved model to: data\names_model_nn
######################
Iteration Number: 3
Losses {'ner': 81.62599249671561}
Duration: 0:00:41.571359
p: 94.11764705882352; f: 94.11764705882352; r: 94.11764705882352
######################
Iteration Number: 4
Losses {'ner': 81.00528897516138}
Duration: 0:00:42.786487
p: 92.57142857142857; f: 93.91304347826087; r: 95.29411764705881
######################
#########

In [18]:
nlp = spacy.load(r"./data/names_model_nn")

In [19]:
no_nn = Person.objects.filter(name="")
no_nn.count()

16163

In [20]:
for x in Person.objects.all():
    doc = nlp("{}".format(x.written_name))
    for ent in doc.ents:
        if ent.label_ == "NN":
#             print(ent.text)
            x.name = "{}".format(ent.text)
            break
    x.save()

# male / female
* trained a male category model with prodigy

In [ ]:
# !pip install spacy==2.0.18 --user rita

In [22]:
nlp = spacy.load(r"C:\Users\pandorfer\Documents\Redmine\prodigy\invs\invs_gender_binary")

In [23]:
for x in Person.objects.all():
    try:
        doc = nlp("{}".format(x.written_name))
    except IndexError:
        doc = None
    if doc is not None:
        if doc.cats['male'] > 0.8:
            x.gender="männlich"
        else:
            x.gender="weiblich"
        x.save()
    

In [ ]:
x.id